# Proyecto de Inteligencia Artificial  
**Tema:** Predictor en Actividades Bancarias  

**Asignatura:** Técnicas en Inteligencia Artificial  
**Docente:** Raúl Ramos Pollán, Jonathan Granda  
**Fecha:** 21-08-2025  

---

## Fuente del Conjunto de Datos  

**Competencia:** [Binary Classification with a Bank Dataset](https://www.kaggle.com/competitions/playground-series-s5e8/overview)  
**Plataforma:** Kaggle – *Tabular Playground Series*  

**Ruta a los datos**
https://www.kaggle.com/competitions/playground-series-s5e8/data
## Objetivo del Banco

Predecir si un cliente **aceptará un depósito a plazo** (y=1) tras una campaña de marketing.  
Con esta predicción el banco puede **priorizar a quién contactar**, **reducir costos de campaña** y **mejorar la tasa de conversión** evitando llamadas innecesarias.

## Qué hace este notebook
- Carga `train.csv` y `test.csv`.
- Limpia y codifica variables (mismo tratamiento para train y test).
- Entrena un **RandomForest** y evalúa (Accuracy, ROC-AUC, PR-AUC, reporte de clasificación).
- Genera `submission.csv` con columnas **`id,y`**.


## 1. Descarga de librerías
**python**

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from google.colab import files
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    average_precision_score,
    classification_report
)
import joblib
from google.colab import files

# Instalar kaggle para poder acceder directamente a los archivos train.csv y test.csv

In [39]:
!pip install -q kaggle

# Cargar json
El profesor debe tener previamente un token de acceso, acceptar previamente la competencia

In [40]:
print("➡️ Selecciona tu archivo kaggle.json")
files.upload()  # Aquí escoges kaggle.json desde tu PC

➡️ Selecciona tu archivo kaggle.json


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"omaralbertotorres","key":"5eccf955249f562d3b538f9334abdbf8"}'}

In [41]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


#Descarga de los datos

In [42]:
!mkdir -p ./datos
!kaggle competitions download -c playground-series-s5e8 -p ./datos
!unzip -o ./datos/*.zip -d ./datos
!ls -lh ./datos


playground-series-s5e8.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ./datos/playground-series-s5e8.zip
  inflating: ./datos/sample_submission.csv  
  inflating: ./datos/test.csv        
  inflating: ./datos/train.csv       
total 101M
-rw-r--r-- 1 root root  15M Jul  5 02:11 playground-series-s5e8.zip
-rw-r--r-- 1 root root 2.7M Jul  5 02:11 sample_submission.csv
-rw-r--r-- 1 root root  21M Jul  5 02:11 test.csv
-rw-r--r-- 1 root root  63M Jul  5 02:11 train.csv


# Lectura de los datos

In [43]:
#--------------------------------------------------------------
# El código lee los archivos *.csv cargados en el paso previo
# Se construye los datasets para entrenamiento y prueba
#--------------------------------------------------------------
train = pd.read_csv("./datos/train.csv")
test  = pd.read_csv("./datos/test.csv")
sample = pd.read_csv("./datos/sample_submission.csv")

print("Train:", train.shape)
print("Test :", test.shape)
print("Sample submission:", sample.shape)

print("\nColumnas de TRAIN:", train.columns.tolist())
print("Columnas de TEST :", test.columns.tolist())
print("Primeras filas de TRAIN:")
print(train.head())


Train: (750000, 18)
Test : (250000, 17)
Sample submission: (250000, 2)

Columnas de TRAIN: ['id', 'age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']
Columnas de TEST : ['id', 'age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']
Primeras filas de TRAIN:
   id  age          job  marital  education default  balance housing loan  \
0   0   42   technician  married  secondary      no        7      no   no   
1   1   38  blue-collar  married  secondary      no      514      no   no   
2   2   36  blue-collar  married  secondary      no      602     yes   no   
3   3   27      student   single  secondary      no       34     yes   no   
4   4   26   technician  married  secondary      no      889     yes   no   

    contact  day month  duration  campaign  pdays  previo

# En este punto tenemos los archivos train.csv y test.csv

In [44]:
# ------------------------------------------------------
# Funciones auxiliares para exploración inicial de datos
# ------------------------------------------------------

def ver_dimensiones(df: pd.DataFrame):
    """Muestra número de filas y columnas."""
    return df.shape

def ver_columnas(df: pd.DataFrame):
    """Devuelve la lista de columnas."""
    return df.columns.tolist()

def ver_tipos(df: pd.DataFrame):
    """Devuelve el tipo de dato de cada columna."""
    return df.dtypes

def ver_nulos(df: pd.DataFrame):
    """Cuenta valores nulos por columna."""
    return df.isnull().sum()

def ver_estadisticas(df: pd.DataFrame):
    """Muestra estadísticas básicas (solo numéricas)."""
    return df.describe()

def load_dataset(path: str) -> pd.DataFrame:
    #return pd.read_excel(path, engine="openpyxl")
    return pd.read_csv("train.csv", sep=',')


# Exploración de Datos – Paso 1

En este primer paso realizamos una **exploración inicial de los datos**

In [45]:
# ---------------------------------------------
# Exploración Inicial del Dataset
# Se imprimen propiedades clave del DataFrame:
# - Dimensiones
# - Columnas
# - Tipos de datos
# - Valores nulos
# - Estadísticas descriptivas
# ---------------------------------------------
def explorar_antes_cleaner_df(df: pd.DataFrame) -> None:
  print("\nInformación del dataset antes del procesamiento\n")
  print(f"Dimensiones del df: {ver_dimensiones(df)}")
  print(f"Columnas del df: {ver_columnas(df)}")
  print(f"Tipos de datos del df: {ver_tipos(df)}")
  print(f"Valores nulos del df: {ver_nulos(df)}")
  print(f"Estadísticas básicas del df: {ver_estadisticas(df)}")
  print("\n")


In [46]:
# -------------------------------------------------------------------------------------------
# Limpieza del Dataset
# Esta función elimina columnas irrelevantes o problemáticas:
# - id: identificador único
# - day, month: variables temporales que pueden inducir ruido
# - duration: fuertemente correlacionada con 'y' (puede inducir fuga de datos)
#
# Parámetros:
# - df (pd.DataFrame): conjunto de datos a limpiar
# - cols_drop (list): columnas a eliminar
# - has_target (bool): True si el DataFrame incluye la columna 'y'
#
# Retorna:
# - Si flag = True: retorna (X, y)
# - Si flag = False: retorna solo X
# -------------------------------------------------------------------------------------------

def clean_df(df: pd.DataFrame, cols_drop: list, flag: bool):
    df_clean = df.drop(columns=cols_drop, errors="ignore")

    if flag:
        y = df_clean["y"]
        X = df_clean.drop(columns=["y"], errors="ignore") #Elimina columna
        return X, y
    else:
        return df_clean  # no hay 'y' en test.csv   #Limpieza del test.csv



In [47]:
# ---------------------------------------------
# Verificación del Dataset Procesado
# Se muestran:
# - Confirmación de limpieza
# - Dimensiones de X (features)
# - Dimensiones de y (target)
# - Primeras filas de X para inspección visual
# ---------------------------------------------
def dataset_cleaner(X: pd.DataFrame, y: pd.DataFrame) ->None:
   print("Dataset despues de limpiado\n")
   print("✅ Dataset limpio creado")
   print("Dimensiones de X:", X.shape)
   print("Dimensiones de y:", y.shape)
   print(X.head())


In [48]:
# ---------------------------------------------------------------------
# Codificación Categórica
# Transforma columnas categóricas (tipo object)
# en valores numéricos utilizando LabelEncoder.
#
#  Requiere que train y test tengan las mismas columnas y categorías.
# ----------------------------------------------------------------------

from sklearn.preprocessing import LabelEncoder

def encode_categoricals(X):
    X_encoded = X.copy()
    for col in X_encoded.select_dtypes(include=["object"]).columns:
        le = LabelEncoder()
        X_encoded[col] = le.fit_transform(X_encoded[col])
    return X_encoded


# División de los Datos

En este paso separamos el dataset en **conjunto de entrenamiento (80%)** y  
**conjunto de validación (20%)**, asegurando que la proporción de clases en la  
variable objetivo se mantenga constante mediante la opción *stratify*.  

De esta forma, el modelo se entrena con una parte de los datos y se evalúa con  
otra parte independiente, evitando el sobreajuste y permitiendo medir su capacidad de generalización.


In [49]:
# ----------------------------------------
#  Verificación de Dimensiones
# ----------------------------------------
import pandas as pd

def mostrar_dimensiones_dataset(
    X_train: pd.DataFrame,
    X_val: pd.DataFrame,
    y_train: pd.Series,
    y_val: pd.Series
) -> None:
    print("Tamaño de X_train:", X_train.shape)
    print("Tamaño de X_val:", X_val.shape)
    print("Tamaño de y_train:", y_train.shape)
    print("Tamaño de y_val:", y_val.shape)


## Entrenamiento del Modelo Random Forest

Se configuró un `RandomForestClassifier` con los siguientes parámetros:

- **n_estimators=300** : número de árboles en el bosque  
- **max_depth=20** : profundidad máxima de cada árbol  
- **n_jobs=-1** : utiliza todos los núcleos disponibles  
- **random_state=42** : asegura reproducibilidad  
- **class_weight="balanced"** : ajusta pesos para manejar clases desbalanceadas  
- **min_samples_leaf=10** : tamaño mínimo de muestras en una hoja  
- **bootstrap=True** : usa muestreo bootstrap para entrenar cada árbol  
- **max_samples=0.7** : cada árbol entrena con el 70% de los datos  
- **oob_score=True** : calcula desempeño con muestras fuera de bolsa  


In [50]:
## -----------------------------------------------------------------------------
#  Entrenamiento del Modelo Random Forest
# ------------------------------------------------------------------------------

def modelo_random_forest(X_train, y_train, X_val):
    modelo = RandomForestClassifier(
        n_estimators=300,
        max_depth=20,
        n_jobs=-1,
        random_state=42,
        class_weight="balanced_subsample",
        min_samples_leaf=20,
        min_samples_split=10,
        bootstrap=True,
        max_samples=0.7,
        oob_score=True
    )
    modelo.fit(X_train, y_train)
    y_pred  = modelo.predict(X_val)
    y_proba = modelo.predict_proba(X_val)[:, 1]
    return modelo, y_pred, y_proba


In [51]:
# ----------------------------------------------------------------------------
# Evaluación del Modelo en Validación
# Se calculan las métricas principales:
# - Accuracy , proporción de aciertos globales
# - ROC-AUC , capacidad de discriminación entre clases
# - PR-AUC , área bajo la curva precisión-recall (importante en desbalance)
# - Classification Report , precisión, recall y F1-score por clase
# ----------------------------------------------------------------------------
def mostrar_resultados(
    y_val:pd.DataFrame,
    y_pred:pd.DataFrame,
    y_proba:pd.DataFrame,
    ) -> pd.DataFrame:
   print("Resultados de la predicción\n")
   print("VAL Accuracy:", accuracy_score(y_val, y_pred))
   print("VAL ROC-AUC:", roc_auc_score(y_val, y_proba))
   print("VAL PR-AUC:", average_precision_score(y_val, y_proba))
   print("\nClassification Report:\n", classification_report(y_val, y_pred))



In [52]:
# ------------------------------------------------------------------------------
# Se realiza la predicción con datos del
# archivo test.csv
# ------------------------------------------------------------------------------

def predict_datos_test(modelo,X_test_num):
  y_pred  = modelo.predict(X_test_num)
  y_proba = modelo.predict_proba(X_test_num)[:, 1]
  return y_pred, y_proba

In [53]:
# ------------------------------------------------------------------------------
# Generación de archivo de envío para Kaggle
# Se construye el submission.csv con:
# - id,  columna original del test.csv
# - y,   predicciones del modelo (0/1)
# ------------------------------------------------------------------------------

def generar_submission(y_pred, ids, ruta_salida="sample_submission.csv"):
    submission = pd.DataFrame({"id": ids, "y": y_pred})
    submission.to_csv(ruta_salida, index=False)
    files.download(ruta_salida)
    return submission


In [54]:
# ------------------------------------------------------------------------------
#
#    Divide X e y en conjuntos de entrenamiento y validación,
#    manteniendo la proporción de clases (stratify=y).

#    Parámetros
#
#    X : Variables independientes
#    y : Variable objetivo
#    test_size : float (default=0.2), Proporción para validación
#    random_state : int (default=42), Semilla para reproducibilidad
#    Retorna
#    X_train, X_val, y_train, y_val
# ------------------------------------------------------------------------------
def split_data(X, y, test_size=0.2, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

## Exportar modelo

In [55]:
# ------------------------------------------------------------------------------
# Funcion para descargar modelo entrenado
# ------------------------------------------------------------------------------
def save_model(modelo):
    filename = "modelo_entrenado.pkl"
    joblib.dump(modelo, filename)
    files.download(filename)
    print(f"Modelo guardado y descargado como '{filename}'")


In [56]:
# ------------------------------------------------------------------------------
# Funcion para construir lista de objetos json para pruebas
# ------------------------------------------------------------------------------
def construir_json_desde_csv(df, n=100, columnas_excluir=None):
    if columnas_excluir:
        columnas_a_eliminar = [col for col in columnas_excluir if col in df.columns]
        df = df.drop(columns=columnas_a_eliminar)
    muestras = df.sample(n=n, random_state=42)
    return [{"data": row.to_dict()} for _, row in muestras.iterrows()]



In [57]:
# ------------------------------------------------------------------------------
#  Programa principal
# ------------------------------------------------------------------------------
def main():
    df = train
    df_test = test

    ids = df_test["id"]    # Guardar IDs para el submission

    # Columnas a eliminar
    cols_drop = ["id", "day", "month", "duration"]

    # Exploración inicial del dataset
    explorar_antes_cleaner_df(df)

    # Preprocesamiento
    X, y        = clean_df(df, cols_drop, flag=True)
    X_test      = clean_df(df_test, cols_drop, flag=False)
    X_num       = encode_categoricals(X)
    X_test_num  = encode_categoricals(X_test)

    # Dataset de entrenamiento limpio
    dataset_cleaner(X_num, y)

    # División train/val
    X_train, X_val, y_train, y_val = split_data(X_num, y)

    # Entrenar y validar
    modelo, y_pred_val, y_proba_val = modelo_random_forest(X_train, y_train, X_val)

    # Mostrar resultados
    mostrar_resultados(y_val, y_pred_val, y_proba_val)

    # Predicción en test y submission
    y_pred_test, y_proba = predict_datos_test(modelo, X_test_num)
    generar_submission(y_pred_test, ids, ruta_salida="sample_submission.csv")

    return modelo, X_test


if __name__ == "__main__":
    modelo, X_test = main()
    save_model(modelo)
    print(X_test.head())



Información del dataset antes del procesamiento

Dimensiones del df: (750000, 18)
Columnas del df: ['id', 'age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']
Tipos de datos del df: id            int64
age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y             int64
dtype: object
Valores nulos del df: id           0
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64
Estadísticas básicas de

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Modelo guardado y descargado como 'modelo_entrenado.pkl'
   age            job  marital  education default  balance housing loan  \
0   32    blue-collar  married  secondary      no     1397     yes   no   
1   44     management  married   tertiary      no       23     yes   no   
2   36  self-employed  married    primary      no       46     yes  yes   
3   58    blue-collar  married  secondary      no    -1380     yes  yes   
4   28     technician   single  secondary      no     1950     yes   no   

    contact  campaign  pdays  previous poutcome  
0   unknown         1     -1         0  unknown  
1  cellular         2     -1         0  unknown  
2  cellular         2     -1         0  unknown  
3   unknown         1     -1         0  unknown  
4  cellular         1     -1         0  unknown  
